In [1]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tweepy
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import os
import nltk
from collections import Counter
import time
import sys
import pandas as pd
import re

consumer_key = os.getenv("TWITTER_PUBLIC_API")
consumer_secret = os.getenv("TWITTER_SECRET_KEY")
nltk.download('vader_lexicon')

addl_stopwords = [',','`', '', 'rt', 'http', 'https', 'RT', 'BTC', 'bitcoin', 'ETH', 'LTC', 'XRP', 'co', 'crypto', 'blockchain', 'cryptocurrency', 'cripto', 'litecoin']

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\cscat\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
def tokenizer(text):
    """Tokenizes text."""
    text = word_tokenize(text)
    text = [word.lower() for word in text]
    regex = re.compile("[^a-zA-Z ]")
    text = [regex.sub('', word) for word in text]
    sw = set(stopwords.words('english') + addl_stopwords)
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text]
    clean_text = [word for word in text if word not in sw]
    return clean_text

def token_count(tokens, N=10):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)


# Functions for Twitter


def get_tweets_list(topic_of_tweet, num_of_tweets):
    '''
    Returns a dataframe of the most recent 'N' tweets from Twitter tokenized and counted.
    
    Arguements: `topic_of_tweet` : str; what hashtag is being searched 
                'num_of_tweets' : int; how many tweet do you want returned
    '''
    text,time, word_list, word_count=[],[],[],[]
    auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
    api = tweepy.API(auth)
    for tweet in tweepy.Cursor(api.search, q=topic_of_tweet, tweet_mode='extended').items(num_of_tweets):
        text.append(tweet.full_text),
        time.append(tweet.created_at)
    tweets_df = pd.DataFrame({'Tweet':text}, index=time)
    [word_list.append(tokenizer(text)) for text in tweets_df.Tweet]
    tweets_df['Tokens'] = word_list
    [word_count.append(token_count(token)) for token in tweets_df.Tokens]
    tweets_df['Word_Count'] = word_count
    
    return tweets_df

def twitter_sent_analysis(tweet_df):    
    tweet_sentiments, comp, pos, neg, neu = [],[],[],[],[]
    analyzer = SentimentIntensityAnalyzer()
    for tweet in tweet_df.Tweet:
        sentiment = analyzer.polarity_scores(tweet),
        comp.append(sentiment[0]["compound"]),
        pos.append(sentiment[0]["pos"]),
        neg.append(sentiment[0]["neg"]),
        neu.append(sentiment[0]["neu"]),
  
    tweet_df['Compound'] = comp
    tweet_df['Positive'] = pos
    tweet_df['Negative'] = neg
    tweet_df['Neutral'] = neu

    return tweet_df

def count(df):
    '''
    Takes a DataFrame with a "compund" column and returns a basic count of positive, neutral, and negative sentiment in a dict format
    '''
    positive_count, negative_count, neutral_count = 0,0,0
    for i in df['Compound']:
        if i >= 0.05:
            positive_count += 1
        elif i <= -0.05:
            negative_count += 1
        else:
            neutral_count += 1
    count={
        'Positive Tweets': positive_count,
        'Neutral Tweets': neutral_count,
       'Negavtive Tweets': negative_count
    }
    return count

def get_twitter_scores(topic_of_tweet, num_of_tweets):
    df = get_tweets_list(topic_of_tweet, num_of_tweets)
    df = twitter_sent_analysis(df)
    
    return df

In [3]:
def twitter_score(df):    
    scores = []
    for ico in df.Name:
        search_term = '#' + str(ico)
        print(f"Searching and Scoring {search_term}")
        tweet_df = get_twitter_scores(search_term, 20)
        score = {ico :{
                'Compound' : tweet_df.Compound.mean(),
                'Positive' : tweet_df.Positive.mean(),
                'Negative' : tweet_df.Negative.mean(),
                'Neutral' : tweet_df.Neutral.mean(),
        }}
        scores.append(score)
        print(f"{ico} scored")
    print(f"Scoring of {len(scores)} tweet concluded, creating dataframe")
    
    return scores

In [4]:
def progressbar(it, prefix="", size=60, file=sys.stdout):
    count = len(it)
    def show(j):
        x = int(size*j/count)
        file.write("%s[%s%s] %i/%i\r" % (prefix, "#"*x, "."*(size-x), j, count))
        file.flush()        
    show(0)
    for i, item in enumerate(it):
        yield item
        show(i+1)
    file.write("\n")
    file.flush()

In [19]:
def twitter_df_score(df, N): 
    '''
    Scores an entire Dataframe of coins based on the last 'N' tweets.  Returns a dataframe of scores with a 
    '''
    scores = []
    df2 = pd.DataFrame
    for name in df.Name.values:
        search_term = str(name)
        print(f"Searching and Scoring {search_term}")
        tweet_df = get_twitter_scores(search_term, N)
        score = {name :{
                'Compound' : tweet_df.Compound.mean(),
                'Positive' : tweet_df.Positive.mean(),
                'Negative' : tweet_df.Negative.mean(),
                'Neutral' : tweet_df.Neutral.mean(),
        }}
        scores.append(score)
        print(f"{name} scored")
        for i in progressbar(range(10), "Waiting for Twitter Rate Limit: ", 40):
            time.sleep(4.5) # any calculation you need
    print(f"Scoring of {len(scores)} tweet concluded, creating dataframe")

    return scores

In [20]:
df = pd.read_csv('../../data/cleandata/success_df.csv')

In [21]:
twitter_df = twitter_df_score(df, 50)
    

Searching and Scoring  Ethereum
 Ethereum scored
Waiting for Twitter Rate Limit: [########################################] 10/10
Searching and Scoring  Binance Coin
 Binance Coin scored
Waiting for Twitter Rate Limit: [########################################] 10/10
Searching and Scoring  EOS
 EOS scored
Waiting for Twitter Rate Limit: [########################################] 10/10
Searching and Scoring  Cardano
 Cardano scored
Waiting for Twitter Rate Limit: [########################################] 10/10
Searching and Scoring  Huobi Token
 Huobi Token scored
Waiting for Twitter Rate Limit: [########################################] 10/10
Searching and Scoring  Tezos
 Tezos scored
Waiting for Twitter Rate Limit: [########################################] 10/10
Searching and Scoring  Cosmos
 Cosmos scored
Waiting for Twitter Rate Limit: [########################################] 10/10
Searching and Scoring  Dash
 Dash scored
Waiting for Twitter Rate Limit: [########################

In [22]:
twitter_df

[{' Ethereum': {'Compound': 0.14712200000000003,
   'Positive': 0.06711999999999999,
   'Negative': 0.01278,
   'Neutral': 0.9200999999999998}},
 {' Binance Coin': {'Compound': 0.08145400000000003,
   'Positive': 0.0246,
   'Negative': 0.00598,
   'Neutral': 0.9693999999999997}},
 {' EOS': {'Compound': 0.075546,
   'Positive': 0.034760000000000006,
   'Negative': 0.009,
   'Neutral': 0.95624}},
 {' Cardano': {'Compound': 0.10752,
   'Positive': 0.059,
   'Negative': 0.018420000000000002,
   'Neutral': 0.9225799999999998}},
 {' Huobi Token': {'Compound': 0.320004,
   'Positive': 0.11982,
   'Negative': 0.00584,
   'Neutral': 0.87434}},
 {' Tezos': {'Compound': 0.17607999999999996,
   'Positive': 0.08024,
   'Negative': 0.011280000000000002,
   'Neutral': 0.9085200000000002}},
 {' Cosmos': {'Compound': 0.027052,
   'Positive': 0.029759999999999998,
   'Negative': 0.01646,
   'Neutral': 0.95378}},
 {' Dash': {'Compound': -0.03234,
   'Positive': 0.0305,
   'Negative': 0.06132000000000001,

In [71]:
twitter_scores_df = pd.DataFrame()
for key in twitter_df:
    x = pd.DataFrame(key).T
    twitter_scores_df = twitter_scores_df.append(x)
twitter_scores_df.head()

,Compound,Negative,Neutral,Positive
Ethereum,0.147122,0.01278,0.92010,0.06712
Binance Coin,0.081454,0.00598,0.96940,0.02460
EOS,0.075546,0.00900,0.95624,0.03476
Cardano,0.107520,0.01842,0.92258,0.05900
Huobi Token,0.320004,0.00584,0.87434,0.11982


In [72]:
twitter_scores_df.to_csv('../../data/cleandata/twitter_scores.csv')